### Measuring and Evaluating Risk
February 2025

*Imports*

In [108]:
import pandas as pd 
import numpy as np
import yfinance as yf
import plotly.graph_objects as go
from datetime import date
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

*Data*

In [109]:
etfs = {
    "XLK": "Technology",
    "XLV": "Health Care",
    "XLF": "Financial",
    "XLE": "Energy",
    "XLI": "Industrial",
    "XLY": "Consumer Discretionary",
    "XLP": "Consumer Staples",
    "XLB": "Materials",
    "XLU": "Utilities",
    "XLRE": "Real Estate"
}

start_date, end_date = date(2020,1,1), date(2025,1,1)

prices = yf.download(tickers=list(etfs.keys()),start = start_date, end = end_date)['Close']

returns = prices.pct_change().dropna()

market_prices = yf.download(tickers=['SPY'],start = start_date, end = end_date)['Close']
market_returns = market_prices.pct_change().dropna()

[*********************100%***********************]  10 of 10 completed
[*********************100%***********************]  1 of 1 completed


### **Risk Statistics**

In [110]:
portfolio_returns = pd.DataFrame(returns.mean(axis = 1),columns=['daily_returns'])
portfolio_returns['cumulative_returns'] = (1+portfolio_returns['daily_returns']).cumprod()

#### 1. Volatility

In [111]:
daily_vol = portfolio_returns['daily_returns'].std()
annual_vol = daily_vol*np.sqrt(252)
print(daily_vol,annual_vol)

0.01302930672706076 0.2068338321323022


In [169]:
vol = pd.DataFrame([daily_vol,annual_vol]).T
vol.columns = ['Daily Volatility','Annualized Volatility']
vol

,Daily Volatility,Annualized Volatility
0,0.013029,0.206834


#### 2. Drawdown and Maximum Drawdown

In [112]:
portfolio_returns['DD_t'] = (portfolio_returns['cumulative_returns'] / portfolio_returns['cumulative_returns'].cummax()) - 1
MMD = min(portfolio_returns['DD_t'])

In [183]:
portfolio_returns.index = pd.to_datetime(portfolio_returns.index).date

In [193]:
fig = go.Figure()


fig.add_trace(
    go.Scatter(
        x = portfolio_returns.index,
        y = portfolio_returns['cumulative_returns'] - 1,
        name = 'Cumulative Returns'
    )
)

fig.add_trace(
    go.Scatter(
        x = portfolio_returns.index,
        y = portfolio_returns['DD_t'],
        name = 'DD_t'
    )
)


fig.add_vline(
    x=portfolio_returns['DD_t'].idxmin(), 
    line_dash="dot",  # Dotted line
    line_color="red", # You can change the color as needed
)

fig.update_layout(
    title = 'Portfolio cumulative returns and drawdown',
    xaxis_title = 'Date',
    yaxis_title = '%',
    xaxis=dict(title_standoff=3),
    template = 'plotly_white',
    margin=dict(l=10, r=10, t=50, b=10),
    legend=dict(orientation="h",yanchor="top",y=-0.1,xanchor="center",x=0.5),
    width = 800,height = 400,
)

fig.show()

#### 3. VaR and CVaR

In [113]:
VaR95 = -np.percentile(portfolio_returns['daily_returns'], 95)
CVaR = -portfolio_returns[portfolio_returns['daily_returns'] <= VaR95]['daily_returns'].mean()

In [164]:
stats = pd.DataFrame([VaR95,CVaR]).T
stats.columns = ['VaR','CVaR']
(stats*100).T

,0
VaR,-1.727854
CVaR,3.013418


### **Risk Models**

#### 1. Beta

In [114]:
# Calculate Beta
beta = portfolio_returns['daily_returns'].cov(market_returns['SPY']) / market_returns['SPY'].var()

# Rolling Beta
beta_rolling = portfolio_returns['daily_returns'].rolling(120).cov(market_returns['SPY']) / market_returns['SPY'].rolling(120).var()

print(round(beta,2))

0.94


*Plot*

In [115]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x = beta_rolling.dropna().index,
        y = beta_rolling.dropna()
    )
)

fig.update_layout(
    title = 'Rolling Beta (120days)',
    template = 'plotly_white',
    yaxis_title = 'Beta',
    xaxis_title = 'Date'
)
fig.show()

#### 2. Factor Models

In [116]:
# Read
data = pd.read_csv('FamaFrench3.csv',skiprows=4,index_col=0)
data = data.iloc[:-1]
data.index = pd.to_datetime(data.index)


# Clean
df = data.merge(portfolio_returns,left_index=True,right_index=True)
df['RF'] = df['RF'] / 100 / 252
df['excess_returns'] = df['daily_returns'] - df['RF']

# Fit Model
X = df[['Mkt-RF','SMB','HML']] / 100
y = df[['excess_returns']]

model = LinearRegression()
model.fit(X, y)
y_pred = model.predict(X)

print(model.coef_)
print(r2_score(y,y_pred))

[[ 0.91864988 -0.07334452  0.2616346 ]]
0.9531333370697259


In [144]:
pd.DataFrame(model.coef_,columns= X.columns)

,Mkt-RF,SMB,HML
0,0.91865,-0.073345,0.261635


#### 3. Statistical Factor Models

In [ ]:
scaler = StandardScaler()
returns_scaled = scaler.fit_transform(returns)

# apply pca
pca = PCA()
pca.fit(returns_scaled)

loadings = pca.components_
loadings_df = pd.DataFrame(loadings,columns = [f'PC {i+1}' for i in range(len(returns.columns))])
loadings_df.index = returns.columns

explained_var_ratio = pca.explained_variance_ratio_

*Plots*

In [150]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x = np.arange(1,11,1),
        y = explained_var_ratio,
    )
)

fig.update_layout(
    title = 'ETF Returns Scree Plot',
    xaxis_title = 'Principal Component',
    yaxis_title = 'Explained Variance Ratio',
    yaxis_range = [0,1],
    template = 'plotly_white',
    margin=dict(l=10, r=10, t=50, b=10),
    width = 800,height = 400,
)
fig.show()

In [151]:
fig = go.Figure()

loadings_df = loadings_df[['PC 1','PC 2']]

for stock in loadings_df.index:
    fig.add_trace(go.Bar(
        x=loadings_df.columns, 
        y=loadings_df.loc[stock], 
        name=stock  
    ))

fig.update_layout(
                  showlegend=True,
                  barmode='group',
                  margin=dict(l=10, r=10, t=50, b=10),
                  legend=dict(orientation="h",yanchor="top",y=-0.1,xanchor="center",x=0.5),
                  width = 800,height = 400,
                  xaxis_title = 'Principal Components',
                  yaxis_title = 'Weight',
                  xaxis=dict(title_standoff=3),
                  title = 'ETF Return PCA Loadings',
                  template = 'plotly_white'
                )

fig.show()